In [2]:
import numpy as np
import torch
import torch.nn as nn
import sys
sys.path.append("../../")
from vi_rnn.vae import VAE
from vi_rnn.train import train_VAE
from torch.utils.data import DataLoader
from vi_rnn.datasets import RDM, RDM_Teacher
from py_rnn.model import RNN, predict
from py_rnn.train import train_rnn
from py_rnn.train import save_rnn, load_rnn
from vi_rnn.utils import *
import matplotlib.pyplot as plt
from vi_rnn.saving import save_model, load_model
from py_rnn.default_params import get_default_params

%matplotlib inline

In [3]:
train_teacher =False  # load already trained teacher model
data_dir = "../../data/student_teacher/"  # store inferred model
model_dir = "../../models/students/"  # store teacher RNN
cuda = True  # toggle if GPU is available

In [4]:
# initialise teacher RNN
model_params, training_params = get_default_params(n_rec=60)
training_params["n_epochs"] = 1000
training_params["l2_rates_reg"] = 0.1
model_params['rank']= 1
rnn_reach = RNN(model_params)

In [ ]:
# Plot an example trial for teacher RNN

task_params_teacher = {
    "name": "rdm",
    "fixation_dur":5,
    "stimulus_dur":40,
    "delay_dur":5,
    "response_dur":1,
    "std":.1,
    "scale":.1}
rdm = RDM(task_params_teacher)
stimulus, targets, mask=rdm.__getitem__(0)
plt.plot(stimulus, c='b', label = 'stimulus')
plt.plot(targets, c='r', label = 'target')
plt.plot(mask, ls='--', c='k', label = 'mask')
plt.legend()

In [6]:
# train or load teacher RNN
if train_teacher:
    losses, reg_losses = train_rnn(
        rnn_reach, training_params, rdm, sync_wandb=False
    )
    save_rnn(
        data_dir + "rdm_rnn2",
        rnn_reach,
        model_params,
        task_params_teacher,
        training_params,
    )
else:
    rnn_reach, model_params, task_params_teacher, training_params = load_rnn(
        data_dir + "rdm_rnn"
    )

In [7]:
# Increase response duration -> makes it easier for student to learn fixed points
task_params_teacher["response_dur"]=50

In [ ]:
# plot example output
inp = []
for i in range(6):
    stim,_,_ = rdm[i]
    inp.append(stim)
inp = torch.stack(inp)
rates, pred = predict(rnn_reach, inp)
fig, axs = plt.subplots(2, figsize=(4, 2))
axs[0].plot(pred[0, :, :])
axs[1].set_xlabel("timesteps")
axs[1].plot(rnn_reach.rnn.nonlinearity(torch.from_numpy(rates[0])));

In [9]:
# Extract weights
U, V, B = extract_orth_basis_rnn(rnn_reach)
I = rnn_reach.rnn.w_inp.detach()


In [15]:
# Generate data using teacher RNN, used for training student RNN
batch_size = 4
task_params = {
    "name": "rdm",
    "n_neurons": rnn_reach.rnn.N,
    "out": "currents",
    "r0": 0,
    "R_x": 0.1,
    "R_z": 0.05,
    "n_trials": 400,
}
task = RDM_Teacher(task_params, task_params_teacher, U, V, B, I, decay=0.9)
data_loader = DataLoader(task, batch_size=batch_size, shuffle=True)

In [ ]:
# Plot example latents of teacher RNN
colors = ["darkred", "firebrick", "tomato", "darkturquoise", "teal", "darkslategrey"]
for tr_i in range(6):
    latent_code = task.latents[:, tr_i]
    v = task.v[tr_i]
    stim = task.stim[tr_i]
    plt.plot(latent_code[0], v[:,0],color=colors[tr_i%6])
    plt.xlabel(r"$z$")
    plt.ylabel(r"$v$")

In [ ]:
# Initialise VI / student setup

dim_z = 1
dim_N = task_params["n_neurons"]
dim_x = task_params["n_neurons"]
dim_u = 1
bs = 10
cuda = True
n_epochs = 500
wandb = False
# initialise encoder
enc_params = {"obs_grad": True, "init_scale": 0.1}

# initialise prior
prior_params = {
    "train_noise_x": True,
    "train_noise_z": True,
    "train_noise_z_t0": True,
    "init_noise_z": 0.1,
    "init_noise_z_t0": 1,
    "init_noise_x": 0.1,
    "scalar_noise_z": "Cov",
    "scalar_noise_x": False,
    "scalar_noise_z_t0": "Cov",
    "identity_readout": True,
    "activation": "relu",
    "exp_par": True,
    "shared_tau": 0.7,
    "readout_from": task_params["out"],
    "train_obs_bias": False,
    "train_obs_weights": False,
    "train_latent_bias": False,
    "train_neuron_bias": True,
    "orth": False,
    "m_norm": False,
    "weight_dist": "uniform",
    "weight_scaler": 1,  # /dim_N,
    "initial_state": "trainable",
    "out_nonlinearity": "identity"
}


training_params = {
    "lr": 1e-3,
    "lr_end": 1e-5,
    "opt_eps": 1e-8,
    "n_epochs": n_epochs,
    "grad_norm": 0,
    "bootstrap": False,
    "eval_epochs": 50,
    "batch_size": bs,
    "cuda": cuda,
    "smoothing": 20,
    "freq_cut_off": 10000,
    "sim_obs_noise": 0,
    "sim_latent_noise": 1,
    "k": 64,
    "loss_f": "opt_VGTF",
    "resample": "systematic",  # , multinomial or none"
    "run_eval": True,
    "smooth_at_eval": False,
    "sim_v":True,
}


VAE_params = {
    "dim_x": dim_x,
    "dim_z": dim_z,
    "dim_u": dim_u,
    "dim_N": dim_N,
    "enc_architecture": "Inv_Obs",
    "enc_params": enc_params,
    "prior_architecture": "PLRNN",
    "prior_params": prior_params,
    "causal": True,
}
vae = VAE(VAE_params)

In [ ]:
# Train

train_VAE(
    vae,
    training_params,
    task,
    sync_wandb=wandb,
    out_dir=model_dir,
    fname=None,
    store_train_stats=False,
)

In [ ]:
save_model(vae, training_params, task_params, name=model_dir + "rdm_student2")